In [1]:
# Google Colab-only setup. No need to run this cell in other environments.
use_colab = True

if use_colab:
    # Mount my Google Drive root folder
    from google.colab import drive
    drive.mount('/content/drive')

    # cd to bayesian-dl-experiments directory
    %cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
    !ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/bayesian-dl-experiments
datasets_files	experiments.ipynb  LICENSE  README.md  ronald_bdl


## Experiment Setup

In [0]:
import torch
import numpy as np

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(682)
np.random.seed(682)

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')
    torch.backends.cudnn.deterministic = True
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = False
    use_pin_memory=True # Faster Host to GPU copies with page-locked memory
else:
    torch_device = torch.device('cpu')
    use_pin_memory=False

# Dataset to use
dataset_name = 'yacht'

# Training set size
dataset_train_size = 0.8

# L2 regularization strength
reg_strength = 0.01

# Epochs
n_epochs = 100

# Number of different data splits to try
n_splits = 20

# Data batch sizes
n_training_batch = 10

# Number of test predictions (for each data point)
n_predictions = 10000

## Prepare data

### Get the data as a torch Dataset object

In [3]:
from torch.utils.data import random_split, DataLoader
from ronald_bdl import datasets

if dataset_name == 'MNIST':
    dataset = datasets.MNIST(root_dir='./datasets_files', transform=None, download=True)
else:
    dataset = datasets.UCIDatasets(dataset_name, root_dir='./datasets_files', transform=None, download=True)

# Set the training/test set sizes
train_size = int(dataset_train_size * len(dataset))
test_size = len(dataset) - train_size
    
# Print the size of the dataset
print("dataset size = " + str(dataset.data.shape))
print("training set size = " + str(train_size))
print("testing set size = " + str(test_size))

Using downloaded and verified file: ./datasets_files/yacht/yacht_hydrodynamics.data
dataset size = torch.Size([308, 7])
training set size = 246
testing set size = 62


## Define network

In [4]:
from ronald_bdl import models

network = models.FCNetMCDropout(
    input_dim=len(dataset.features), 
    output_dim=len(dataset.targets),
    hidden_dim=100,
    n_hidden=2,
    dropout_rate=0.01,
)

# Send the whole model to the selected torch.device
network.to(torch_device)

# Print the network structure
print(network)

FCNetMCDropout(
  (input): Linear(in_features=6, out_features=100, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
  )
  (output): Linear(in_features=100, out_features=1, bias=True)
)


## Train the network

### Setup

In [0]:
from torch import nn, optim

# Model to train mode
network.train()

# Mean Squared Error for loss function to minimize
objective = nn.MSELoss()

rmse_non_mc, rmse_mc, test_lls_mc = [], [], []

### Train/test the model

In [6]:
import time

for s in range(n_splits):

    # Prepare new train-test split
    train, test = random_split(dataset, lengths=[train_size, test_size])
    train_loader = DataLoader(train, batch_size=n_training_batch, pin_memory=use_pin_memory)
    if use_pin_memory: test.dataset.data.pin_memory()
    
    # Adam optimizer
    # https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam
    # NOTE: Need to set L2 regularization from here
    optimizer = optim.Adam(
        network.parameters(),
        lr=0.01,
        weight_decay=reg_strength, # L2 regularization
    )
    
    """
    Training
    """
    # Record training start time (for this split)
    tic = time.time()

    for epoch in range(n_epochs): # loop over the dataset multiple times

        print("Running split " + str(s) + ", epoch " + str(epoch))

        for i, data in enumerate(train_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, targets = data

            # Store the batch to torch_device's memory
            inputs = inputs.to(torch_device)
            targets = targets.to(torch_device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = network(inputs)

            loss = objective(outputs, targets)
            loss.backward()

            optimizer.step()
            
    # Record training end time
    toc = time.time()
            
    """
    Testing
    """
    # Model to eval mode
    network.eval()

    # Get the test data
    inputs, targets = test.dataset[test.indices]

    # Store the batch to torch_device's memory
    inputs = inputs.to(torch_device)
    targets = targets.to(torch_device)

    predictions, mean, var, metrics = network.mc_predict(inputs, n_predictions,
                                                         y_test=targets, reg_strength=reg_strength)

    """
    Print results
    """
    print()
    print("Running split " + str(s) + " test:")
    print("Mean = " + str(mean))
    print("Variance = " + str(var))

    # Print and store additional metrics
    if len(metrics) > 0:
        for key, value in metrics.items():
            print(str(key) + " = " + str(value))

            if key == 'rmse_mc': rmse_mc.append(value.item())
            elif key == 'rmse_non_mc': rmse_non_mc.append(value.item())
            elif key == 'test_ll_mc': test_lls_mc.append(value.item())

    # Report the total training time
    print("Split " + str(s) + " training time = " + str(toc - tic) + " seconds")
    print()

Running split 0, epoch 0
Running split 0, epoch 1
Running split 0, epoch 2
Running split 0, epoch 3
Running split 0, epoch 4
Running split 0, epoch 5
Running split 0, epoch 6
Running split 0, epoch 7
Running split 0, epoch 8
Running split 0, epoch 9
Running split 0, epoch 10
Running split 0, epoch 11
Running split 0, epoch 12
Running split 0, epoch 13
Running split 0, epoch 14
Running split 0, epoch 15
Running split 0, epoch 16
Running split 0, epoch 17
Running split 0, epoch 18
Running split 0, epoch 19
Running split 0, epoch 20
Running split 0, epoch 21
Running split 0, epoch 22
Running split 0, epoch 23
Running split 0, epoch 24
Running split 0, epoch 25
Running split 0, epoch 26
Running split 0, epoch 27
Running split 0, epoch 28
Running split 0, epoch 29
Running split 0, epoch 30
Running split 0, epoch 31
Running split 0, epoch 32
Running split 0, epoch 33
Running split 0, epoch 34
Running split 0, epoch 35
Running split 0, epoch 36
Running split 0, epoch 37
Running split 0, epoch

### Print statistics

In [7]:
# Copied from DropoutUncertaintyExps repo
print('non-MC RMSE %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(rmse_non_mc), np.std(rmse_non_mc), np.std(rmse_non_mc)/np.sqrt(n_splits),
        np.percentile(rmse_non_mc, 50), np.percentile(rmse_non_mc, 25), np.percentile(rmse_non_mc, 75)))

print('MC RMSE %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(rmse_mc), np.std(rmse_mc), np.std(rmse_mc)/np.sqrt(n_splits),
        np.percentile(rmse_mc, 50), np.percentile(rmse_mc, 25), np.percentile(rmse_mc, 75)))

print('MC Test Log-likelihood %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(test_lls_mc), np.std(test_lls_mc), np.std(test_lls_mc)/np.sqrt(n_splits), 
        np.percentile(test_lls_mc, 50), np.percentile(test_lls_mc, 25), np.percentile(test_lls_mc, 75)))

non-MC RMSE 2.502890 +- 1.187413 (stddev) +- 0.265514 (std error), median 2.162477 25p 1.748644 75p 2.531187 

MC RMSE 2.073684 +- 1.153118 (stddev) +- 0.257845 (std error), median 1.707645 25p 1.427558 75p 2.364977 

MC Test Log-likelihood -3.254351 +- 0.035605 (stddev) +- 0.007962 (std error), median -3.242203 25p -3.254255 75p -3.236097 

